<a href="https://colab.research.google.com/github/Yewon9/Emotion_Recognition_STT/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install transformers
!pip install sentencepiece

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
import re
import pandas as pd

# 일반 음성 EDA

In [13]:
emotion_text = pd.read_csv('emotion_text_1.csv')
emotion_text

,Emotion,Path,Text
0,disgust,/content/drive/MyDrive/project/data_aihub/4/5e...,고등학교 동창인데 아 이렇게 더럽게 쓸 줄은 몰랐어
1,sad,/content/drive/MyDrive/project/data_aihub/4/5e...,그럴 시간도 없다
2,neutral,/content/drive/MyDrive/project/data_aihub/4/5e...,그래 고마워
3,angry,/content/drive/MyDrive/project/data_aihub/4/5e...,맨날 그래 얘
4,surprise,/content/drive/MyDrive/project/data_aihub/4/5e...,일주일에 다섯 번을 먹는다니까
...,...,...,...
12674,surprise,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,거기 안에다가 AI 넣으면 사람이야
12675,fear,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,음성을 인식할 수 없음
12676,neutral,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,음성을 인식할 수 없음
12677,neutral,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,그치 그


In [14]:
emotion_text = emotion_text[~emotion_text['Text'].isin(['음성을 인식할 수 없음', 'STT API 오류'])]
emotion_text

,Emotion,Path,Text
0,disgust,/content/drive/MyDrive/project/data_aihub/4/5e...,고등학교 동창인데 아 이렇게 더럽게 쓸 줄은 몰랐어
1,sad,/content/drive/MyDrive/project/data_aihub/4/5e...,그럴 시간도 없다
2,neutral,/content/drive/MyDrive/project/data_aihub/4/5e...,그래 고마워
3,angry,/content/drive/MyDrive/project/data_aihub/4/5e...,맨날 그래 얘
4,surprise,/content/drive/MyDrive/project/data_aihub/4/5e...,일주일에 다섯 번을 먹는다니까
...,...,...,...
12671,fear,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,무서운데
12672,neutral,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,그거 뭐야
12674,surprise,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,거기 안에다가 AI 넣으면 사람이야
12677,neutral,/content/drive/MyDrive/project/KEMDy20/wav/Ses...,그치 그


In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', text)
    return text

In [15]:
emotion_text['cleaned_text'] = emotion_text['Text'].apply(clean_text)

<ipython-input-15-a831aa09bce6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emotion_text['cleaned_text'] = emotion_text['Text'].apply(clean_text)


In [16]:
print(emotion_text[['Text', 'cleaned_text']].head())

                           Text                  cleaned_text
0  고등학교 동창인데 아 이렇게 더럽게 쓸 줄은 몰랐어  고등학교 동창인데 아 이렇게 더럽게 쓸 줄은 몰랐어
1                     그럴 시간도 없다                     그럴 시간도 없다
2                        그래 고마워                        그래 고마워
3                       맨날 그래 얘                       맨날 그래 얘
4              일주일에 다섯 번을 먹는다니까              일주일에 다섯 번을 먹는다니까


In [17]:
def tokenize_text(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

In [18]:
emotion_text['tokenized'] = emotion_text['cleaned_text'].apply(tokenize_text)

<ipython-input-18-1ffc759f5e00>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emotion_text['tokenized'] = emotion_text['cleaned_text'].apply(tokenize_text)


In [19]:
print(emotion_text['tokenized'].head())

0    [input_ids, token_type_ids, attention_mask]
1    [input_ids, token_type_ids, attention_mask]
2    [input_ids, token_type_ids, attention_mask]
3    [input_ids, token_type_ids, attention_mask]
4    [input_ids, token_type_ids, attention_mask]
Name: tokenized, dtype: object


In [20]:
emotion_text.to_csv('emotion_tokenized_data.csv', index = False, encoding = 'utf-8-sig')